In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 20 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

07 March 2023 ; 16:10:03


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,30,0,9.7,1.6,3.6,0.444,...,4.03,115.27,111.36,3.91,3.97,115.34,111.48,3.86,1,1
1,Bobby Portis,PF,27,MIL,53,15,25.8,5.6,11.4,0.494,...,4.03,115.27,111.36,3.91,3.97,115.34,111.48,3.86,0,1
5,Jae Crowder,SF,32,MIL,6,0,17.3,1.7,3.7,0.455,...,4.03,115.27,111.36,3.91,3.97,115.34,111.48,3.86,0,1
7,Joe Ingles,SF,35,MIL,31,0,22.4,2.3,5.6,0.405,...,4.03,115.27,111.36,3.91,3.97,115.34,111.48,3.86,0,1
9,Khris Middleton,SF,31,MIL,21,7,21.8,4.8,11.4,0.418,...,4.03,115.27,111.36,3.91,3.97,115.34,111.48,3.86,0,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Malcolm Brogdon,25.6,55,0,BOS,18.5,2.534545,7.007273,14.8,2,2.7,4.621818,0.319924
1,Bobby Portis,25.8,53,15,MIL,18.1,1.701887,6.498113,13.7,1,1.3,2.939623,0.151675
2,Malik Monk,22.5,61,0,SAC,16.2,0.672131,3.091803,13.6,3,1.3,2.016393,0.114770
3,Christian Wood,26.9,53,17,DAL,20.9,2.011321,6.188679,17.4,7,2.1,3.558491,0.099921
4,Immanuel Quickley,28.0,65,11,NYK,15.4,2.018462,6.307692,13.4,5,0.9,4.036923,0.067233


In [7]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)